In [2]:
# things we need for NLP
import nltk
#from nltk.stem.lancaster import LancasterStemmer
from nltk import stem

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [3]:
# import our chat-bot intents file
import json
with open('intents.4.json') as json_data:
    intents = json.load(json_data)

In [5]:
words = []
classes = []
documents = []
ignore_words = ['?', '', '.', ',', '-', '-ийн', '-ын', '-тай', '-г', '-ийг','-д', '-н', '-ний', '-дээр', 'ч', 'л', 'нь', 'юм', 'бэ', 'вэ', 'юу', 'гэж', 'уу']
my_rule_tuple = ("ы$|ий$|ыг$|ийг$|ны$|ний$|лаа$|лээ$|лоо$|лөө$|даа$|дээ$|доо$|дөө$|уулж$|үүлж$|гдах$|гдэх$|гдох$|гдөх$|лалт$|лэлт$|лолт$|лөлт$|дахад$|дэхэд$|доход$|дөхөд$|уудыг$|үүдийг$|нүүдийг$|нуудыг$|чихлаа$|чихлээ$|чихлоо$|чихлөө$|ид$|ын$|ийн$|уудын$|үүдийн$|тай$|той$|төй$|тэй$|лал$|лэл$|лол$|лөл$|аар$|ээр$|оор$|өөр$|лах$|лэх$|лох$|лөх$|ахад$|эхэд$|оход$|өхөд$|лалд$|лэлд$|лөлд$|лолд$|ууд$|үүд$|даг$|дэг$|дог$|дөг$|ддаг$|ддэг$|ддог$|ддөг$|даж$|дэж$|дож$|дөж$|сэн$|сан$|сөн$|сон$|гах$|гэх$|гох$|гөх$|рах$|рэх$|рох$|рөх$|иас$|иэс$|иос$|иөс$|нд$|нт$|ад$|эд$|од$|өд$|ааж$|ээж$|оож$|өөж$|лаар$|лээр$|лоор$|лөөр$|аг$|эг$|ог$|өг$|уг$|өөг$|ээг$|оог$|үүг$|ууг$|бар$|бэр$|бор$|бөр$|бур$|бүр$|раа$|рээ$|рөө$|рүү$|роо$|руу$|аас$|ээс$|оос$|өөс$|аарх$|ээрх$|оорх$|уурх$|үүрх$|өөрх$|аад$|ээд$|ууд$|үүд$|оод$|өөд$|нууд$|нүүд$|дсан$|дсэн$|дсөн$|дсон$|дах$|дэх$|дох$|дөх$|хлаа$|хлээ$|хлоо$|хлөө$|лахдаа$|лэхдээ$|лохдоо$|лөхдөө$|лдаг$|лдэг$|лж$|аагүй$|ээгүй$|оогүй$|өөгүй$|даггүй$|дэггүй$|доггүй$|дөггүй$|нуудийнхаа$|нүүдийнхээ$|гүй$")
#stemmer = LancasterStemmer()
stemmer = stem.RegexpStemmer(my_rule_tuple, min=6)

# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents", '\n')
print (len(classes), "classes", classes, '\n')
print (len(words), "unique stemmed words", words)

4791 documents 

446 classes ['BackUp   -г хэрхэн хийх вэ?', 'CRM гэж юу вэ', 'Grade -н бүртгэл дээрх зэрэглэл багана юуг илэрхийлдэг вэ?', 'Green Finance систем Green ERP  -н бусад системтэй хэрхэн холбогддог вэ?', 'Green Finance систем гэж юу вэ?', 'Green Finance систем нь хэдэн модультой вэ?', 'GreenMailer   -г хэрхэн тохируулах вэ?', 'GreenMailer гэж юу вэ?', 'GreenMailer\xa0лог\xa0хэсэгт Pipeline гэсэн алдаа гарсан байна. Яах вэ?', 'GreenSystem ажиллуулаад систем нээгдэхгүй байна, яах вэ?', 'Iphone дээр ажиллах уу?', 'Oracle  -н Standard Edition дээр ажиллах уу?', 'Oracle  -н Xpress Edition дээр ажиллах уу?', 'Oracle  -н ямар хувилбарыг дэмжих вэ?', 'RMA захиалга гэж юу вэ', 'SCM гэж юу вэ?', 'SSL дэмжих үү?', 'TimeFData   -г хэрхэн шинэчлэх вэ?', 'TimeFData   -г яаж ажиллуулах вэ?', 'UploadManager дээр файлын жагсаалт гарч ирэхгүй байна?', 'goodbye', 'greeting', 'hours', 'thanks', 'Авлага, өглөгийг дурын excel загвараас импортлох боломжтой юу?', 'Авлагын илүү төлөлт гэж юу вэ?', 

In [6]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [8]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=500, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 299499  | total loss: 0.00169 | time: 7.513s
| Adam | epoch: 500 | loss: 0.00169 - acc: 0.9990 -- iter: 4784/4791
Training Step: 299500  | total loss: 0.00152 | time: 7.528s
| Adam | epoch: 500 | loss: 0.00152 - acc: 0.9991 -- iter: 4791/4791
--
INFO:tensorflow:/storage/Workspaces/workspace3/nlp/ub2/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [9]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [10]:
p = bow("Барааны төлөв гэж юу вэ?", words, True)

print (p)
#print (classes)

found in bag: бараа
found in bag: төлөв
found in bag: гэж
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [11]:
#print(model.predict([p]))

In [12]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [ ]:
print ('')